In [1]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web
from datetime import date as dt

In [2]:
# set parameters for CAPM. Choose your security, benchmark, risk free rate (or proxy for risk free rate), start & end dates for the downloaded data
RISKY_ASSET = 'AXP'
START_DATE = '2013-03-31'
END_DATE = dt.today()

In [3]:
ff_dict = web.DataReader("F-F_Research_Data_Factors",
                         "famafrench",
                         start = START_DATE,
                         end = END_DATE)
ff_dict

{0:          Mkt-RF   SMB   HML    RF
 Date                             
 2013-03    4.03  0.81 -0.19  0.00
 2013-04    1.55 -2.36  0.45  0.00
 2013-05    2.80  1.73  2.63  0.00
 2013-06   -1.20  1.33  0.03  0.00
 2013-07    5.65  1.86  0.57  0.00
 ...         ...   ...   ...   ...
 2022-10    7.83  0.09  8.05  0.23
 2022-11    4.60 -3.40  1.38  0.29
 2022-12   -6.41 -0.68  1.32  0.33
 2023-01    6.65  5.02 -4.05  0.35
 2023-02   -2.58  1.10 -0.80  0.34
 
 [120 rows x 4 columns],
 1:       Mkt-RF    SMB    HML    RF
 Date                            
 2013   35.20   7.75   2.60  0.02
 2014   11.71  -7.86  -1.45  0.02
 2015    0.09  -3.99  -9.65  0.02
 2016   13.30   6.66  22.75  0.20
 2017   21.51  -4.97 -13.51  0.80
 2018   -6.95  -3.21  -9.73  1.83
 2019   28.28  -6.11 -10.34  2.15
 2020   23.66  13.18 -46.56  0.45
 2021   23.56  -3.89  25.53  0.04
 2022  -21.60  -6.82  25.80  1.43,
 'DESCR': 'F-F Research Data Factors\n-------------------------\n\nThis file was created by CMPT_ME_BEM

In [4]:
factor_3_df = ff_dict[0].rename(columns = {"Mkt-RF": "MKT"})\
    .div(100)
factor_3_df.iloc[-6:]

,MKT,SMB,HML,RF
Date,,,,
2022-09,-0.0935,-0.0079,0.0006,0.0019
2022-10,0.0783,0.0009,0.0805,0.0023
2022-11,0.0460,-0.0340,0.0138,0.0029
2022-12,-0.0641,-0.0068,0.0132,0.0033
2023-01,0.0665,0.0502,-0.0405,0.0035
2023-02,-0.0258,0.0110,-0.0080,0.0034


In [5]:
# create data frame of timeseries for asset, benchmark, and risk free rate proxy
df = yf.download([RISKY_ASSET],
                 start=START_DATE,
                 end=END_DATE,
                 progress=False,
                 auto_adjust=True)
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2013-04-01,58.190928,58.475756,57.837056,58.052834,3012600
2013-04-02,58.156399,58.484385,57.914729,58.380810,3921600
2013-04-03,58.571272,58.597241,57.168884,57.350674,5102000
2013-04-04,57.497812,57.965275,57.393936,57.766174,3633900
2013-04-05,56.935134,56.961110,56.069464,56.528275,6898300


In [6]:
# calculate returns
y = df['Close'].resample('M') \
    .last() \
    .pct_change() \
    .dropna()

y.index = y.index.to_period("m")
y.name = "ret"
y.iloc[-5:]

Date
2022-12   -0.062440
2023-01    0.188063
2023-02   -0.005374
2023-03   -0.051957
2023-04   -0.006822
Freq: M, Name: ret, dtype: float64

In [7]:
factor_3_df = factor_3_df.join(y)
factor_3_df["excess_ret"] = ( factor_3_df["ret"] - factor_3_df["RF"])
factor_3_df.iloc[-5:]

,MKT,SMB,HML,RF,ret,excess_ret
Date,,,,,,
2022-10,0.0783,0.0009,0.0805,0.0023,0.104564,0.102264
2022-11,0.0460,-0.0340,0.0138,0.0029,0.061569,0.058669
2022-12,-0.0641,-0.0068,0.0132,0.0033,-0.062440,-0.065740
2023-01,0.0665,0.0502,-0.0405,0.0035,0.188063,0.184563
2023-02,-0.0258,0.0110,-0.0080,0.0034,-0.005374,-0.008774


In [8]:
fama_french_model = smf.ols(formula = "excess_ret ~ MKT + SMB + HML",
                            data = factor_3_df).fit()
print(fama_french_model.summary())

                            OLS Regression Results                            
Dep. Variable:             excess_ret   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.576
Method:                 Least Squares   F-statistic:                     53.90
Date:                Sat, 15 Apr 2023   Prob (F-statistic):           9.11e-22
Time:                        01:27:53   Log-Likelihood:                 194.42
No. Observations:                 118   AIC:                            -380.8
Df Residuals:                     114   BIC:                            -369.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0009      0.004      0.200      0.8